In [4]:
import sys
sys.path.append('../src')

In [5]:
# Importing stock ml libraries
from sklearn import metrics
import torch
from torch.utils.data import DataLoader

In [6]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [7]:
from data_preprocessing import ANOMALY_LABELS, ANOMALY_SUBCATEGORIES, load_data
from custom_dataset import CustomDataset
from model import SequenceClassificationModel
from metrics import custom_classification_report
from losses import loss

In [5]:
# Defining some key variables that will be used later on in the training
MODEL_NAME = "AeroBOT"
MODEL_DIRECTORY = "model_save"


# ABBREVIATION = True
ABBREVIATION = False


LOSS_TYPE = 'BCE'
# LOSS_TYPE = 'BinaryFocal'

BALANCED = False
# BALANCED = True

# LAYERS_TO_UNFREEZE = None
LAYERS_TO_UNFREEZE = [8, 9, 10, 11]

ENCODER_NAME = 'bert-base-uncased'
# ENCODER_NAME = 'NASA-AIML/MIKA_SafeAeroBERT'
# ENCODER_NAME = 'allenai/longformer-base-4096'

# IS_SUBCATEGORY = True
IS_SUBCATEGORY = False


if MODEL_NAME == 'AeroBOT':
    MAX_LEN = 200
elif ENCODER_NAME == 'allenai/longformer-base-4096':
    MAX_LEN = 1024
else:
    MAX_LEN = 512

TRAIN_EFFECTIVE_BATCH_SIZE = 32 # 32 Effective size for NASA
TRAIN_BATCH_SIZE = 32
ACCUMULATION_STEPS = TRAIN_EFFECTIVE_BATCH_SIZE / TRAIN_BATCH_SIZE
VALID_BATCH_SIZE = TRAIN_BATCH_SIZE
EPOCHS = 5 # 5 Epochs for NASA
LEARNING_RATE = 1e-05 * 2 # 0.00002 Rate for NASA

drop the NaN values in Anomaly?

In [6]:
if IS_SUBCATEGORY:
    labels = ANOMALY_SUBCATEGORIES
else:
    labels = ANOMALY_LABELS
    
if ABBREVIATION:
    train_df = load_data("../data/train_data_final.pkl", labels, pp_path="../data/train_data_processed2.pkl")
    test_df = load_data("../data/test_data_final.pkl", labels, pp_path="../data/test_data_processed2.pkl")
else: 
    train_df = load_data("../data/train_data_final.pkl", labels)
    # test_df = load_data("../data/test_data_final.pkl", labels)
    import pandas as pd
    test_df = pd.read_csv("../data/subset_test_data.csv")
    # Define a function to convert lists to numpy arrays
    def str_to_list(string):
        # Remove the square brackets and split the string by spaces
        elements = string.strip('[]').split()
        # Convert each element to an integer and create a list
        return [int(x) for x in elements]

    # Apply the function to the column containing lists
    test_df['labels'] = test_df['labels'].apply(str_to_list)
train_df

,text,labels
ACN,,
1163382,I was the pilot flying performing the takeoff....,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
893734,We had 6 shipments of dry ice for the flight; ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
991883,I have seen a lot of mistakes on every flight ...,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
1590076,It was my first time flying into KEUG and I wa...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1715282,I am writing this report to bring attention to...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...
622204,WE WERE ENRTE IN LNAV AT FL310; 30 MI N OF ATL...,"[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
622205,CLRED BY TWR CTL TO CROSS RWY 8R/26L AT TXWY E...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
661202,WHILE WORKING NUMEROUS CVG AND CMH DEPS AT A C...,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [7]:
test_df

,index,text,labels
0,6922,We took off from ZZZ as usual. We were on the ...,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
1,5110,The student was forced to maneuver our aircraf...,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
2,2764,In our Routine Overnight Hangar we are repeate...,"[1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
3,2880,While searching for our runway in visual condi...,"[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
4,5773,We accepted a release and subsequently departe...,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...
1384,1576,Multiple large potholes with loose gravel on T...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
1385,7319,I fly to RBG often. RBG is surrounded by hilly...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
1386,7546,[I] reported early morning for the last day of...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
1387,10519,CLT Airport: The taxi lighting on the east si...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"


In [8]:
num_labels = len(test_df.labels.iloc[0])
model = SequenceClassificationModel(ENCODER_NAME, num_labels=num_labels, model_name=MODEL_NAME)
model.set_trainable_layers(LAYERS_TO_UNFREEZE)
print(model.to(device))

if ABBREVIATION:
    model.model_name += '_Abbreviated'


if IS_SUBCATEGORY:
    model.model_name += '_Subcategory'


SequenceClassificationModel(
  (l1): AeroBOTSequenceClassification(
    (encoder): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_

In [9]:
# Creating the dataset and dataloader for the neural network
print("TRAIN Dataset: {}".format(train_df.shape))
print("TEST Dataset: {}".format(test_df.shape))

tokenizer = model.tokenizer()
training_set = CustomDataset(train_df, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_df, tokenizer, MAX_LEN)

TRAIN Dataset: (96986, 2)
TEST Dataset: (1389, 3)


In [10]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 2
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 2
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [11]:
loss_fn = loss(model, LOSS_TYPE, BALANCED, training_set, training_loader, device)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)
metrics_dict = {
    "Custom Classification Report": lambda y_true, y_pred: custom_classification_report(y_true, y_pred, labels),
    "Optimization Metric": lambda y_true, y_pred: metrics.f1_score(y_true, y_pred, average='macro', zero_division=0)
}


In [12]:
model.model_name

'AeroBOT_bert-base-uncased_Unfrozen[8, 9, 10, 11]_BCE'

In [13]:
from training import TrainingHandler
# trainer = TrainingHandler(model, optimizer, loss_fn, device, labels, metrics_dict, directory=MODEL_DIRECTORY, base_path=MODEL_NAME)
trainer = TrainingHandler(model, optimizer, loss_fn, device, labels, metrics_dict, directory=MODEL_DIRECTORY)
trainer.train(training_loader, testing_loader, save=True, epochs=EPOCHS)

Model loaded from model_save/AeroBOT_bert-base-uncased_Unfrozen[8, 9, 10, 11]_BCE_epoch_4.pth


In [14]:
# from evaluation import EvaluationHandler
# evaluator = EvaluationHandler(model, device, labels, metrics_dict)
# _, _, thresholds = evaluator.evaluate(training_loader, loss_fn, optimize=True)
# hyperparameters={"thresholds": thresholds}
# loss, metrics_results, thresholds = evaluator.evaluate(testing_loader, loss_fn, optimize=False, hyperparameters=hyperparameters)

# thresholds

In [15]:
# from model_management import ModelManager
# ModelManager.save_results(model.model_name, metrics_results, thresholds, "../out/results-sub.csv")

In [16]:
from evaluation import EvaluationHandler
evaluator = EvaluationHandler(model, device, labels, metrics_dict)
pred = evaluator.evaluate(testing_loader, loss_fn, optimize=True, return_pred=True)


Evaluation - Batch: 44 [  572/ 1389], Time: 4s 38ms/step, Loss: 0.402448
Optimizing Thresholds


tensor([[1., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])

In [21]:
pred_df = pd.read_csv("pred.csv")
# pred_df[model.model_name] = pred.tolist()
# pred_df.to_csv("../out/pred.csv")
pred_df

,Unnamed: 0,index,text,labels,"AeroBOT_bert-base-uncased_Unfrozen[8, 9, 10, 11]_BCE"
0,0,6922,We took off from ZZZ as usual. We were on the ...,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
1,1,5110,The student was forced to maneuver our aircraf...,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2,2764,In our Routine Overnight Hangar we are repeate...,"[1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
3,3,2880,While searching for our runway in visual condi...,"[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
4,4,5773,We accepted a release and subsequently departe...,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...
1384,1384,1576,Multiple large potholes with loose gravel on T...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
1385,1385,7319,I fly to RBG often. RBG is surrounded by hilly...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1386,1386,7546,[I] reported early morning for the last day of...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1387,1387,10519,CLT Airport: The taxi lighting on the east si...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
